# FastText Tutorial

Using large amounts of unannotated plain text, fasttext learns relationships between words automatically. The output are vectors, one vector per word, with remarkable linear relationships that allow us to do things like vec(“king”) – vec(“man”) + vec(“woman”) =~ vec(“queen”), or vec(“Montreal Canadiens”) – vec(“Montreal”) + vec(“Toronto”) resembles the vector for “Toronto Maple Leafs”.

## This tutorial

In this tutorial you will learn how to train and evaluate fasttext models.


## Preparing the Input
Starting from the beginning, gensim’s `fasttext` expects a sequence of sentences as its input. Each sentence is a list of words (utf8 strings):

In [26]:
# import modules & set up logging
import gensim, logging
from gensim.models.fasttext import FastText

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [27]:
from gensim.models.word2vec import LineSentence

# Set file names for train and test data
data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = data_dir + 'lee_background.cor'
lee_data = LineSentence(lee_train_file)

sentences = [['first', 'sentence'], ['second', 'sentence']]
# train fasttext on the two sentences
model = FastText(min_count=1)

# build the vocabulary
model.build_vocab(lee_data)

# train the model
model.train(lee_data, total_examples=model.corpus_count, epochs=model.epochs)

print(model)

2018-06-16 17:40:49,539 : INFO : collecting all words and their counts
2018-06-16 17:40:49,541 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-16 17:40:49,568 : INFO : collected 10781 word types from a corpus of 59890 raw words and 300 sentences
2018-06-16 17:40:49,568 : INFO : Loading a fresh vocabulary
2018-06-16 17:40:49,611 : INFO : min_count=1 retains 10781 unique words (100% of original 10781, drops 0)
2018-06-16 17:40:49,612 : INFO : min_count=1 leaves 59890 word corpus (100% of original 59890, drops 0)
2018-06-16 17:40:49,661 : INFO : deleting the raw counts dictionary of 10781 items
2018-06-16 17:40:49,663 : INFO : sample=0.001 downsamples 37 most-common words
2018-06-16 17:40:49,664 : INFO : downsampling leaves estimated 47821 word corpus (79.8% of prior 59890)
2018-06-16 17:40:50,657 : INFO : estimated required memory for 10781 words, 76621 buckets and 100 dimensions: 47172220 bytes
2018-06-16 17:40:50,661 : INFO : resetting layer weights
2

FastText(vocab=10781, size=100, alpha=0.025)


Keeping the input as a Python built-in list is convenient, but can use up a lot of RAM when the input is large.

Gensim only requires that the input must provide sentences sequentially, when iterated over. No need to keep everything in RAM: we can provide one sentence, process it, forget it, load another sentence…

For example, if our input is strewn across several files on disk, with one sentence per line, then instead of loading everything into an in-memory list, we can process the input file by file, line by line:

In [28]:
# create some toy data to use with the following example
import smart_open, os

if not os.path.exists('./data/'):
    os.makedirs('./data/')

filenames = ['./data/f1.txt', './data/f2.txt']

for i, fname in enumerate(filenames):
    with smart_open.smart_open(fname, 'w') as fout:
        for line in sentences[i]:
            fout.write(line + '\n')

In [29]:
from smart_open import smart_open
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in smart_open(os.path.join(self.dirname, fname), 'rb'):
                yield line.split()

In [30]:
sentences = MySentences('./data/') # a memory-friendly iterator
print(list(sentences))

[[b'first'], [b'sentence'], [b'second'], [b'sentence']]


In [31]:
sentences = [[str(m) for m in s] for s in sentences]

In [32]:
# generate the FastText model
model = FastText(sentences, min_count=1)

2018-06-16 17:41:09,078 : INFO : collecting all words and their counts
2018-06-16 17:41:09,079 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-16 17:41:09,081 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2018-06-16 17:41:09,082 : INFO : Loading a fresh vocabulary
2018-06-16 17:41:09,082 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-06-16 17:41:09,083 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-06-16 17:41:09,084 : INFO : deleting the raw counts dictionary of 3 items
2018-06-16 17:41:09,085 : INFO : sample=0.001 downsamples 3 most-common words
2018-06-16 17:41:09,086 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-06-16 17:41:09,087 : INFO : estimated required memory for 3 words, 87 buckets and 100 dimensions: 39596 bytes
2018-06-16 17:41:09,088 : INFO : resetting layer weights
2018-06-16 17:41:09,550 : INFO : Total number of ng

In [33]:
print(model)
print(model.wv.vocab)

FastText(vocab=3, size=100, alpha=0.025)
{"b'first'": <gensim.models.keyedvectors.Vocab object at 0x113a24ac8>, "b'sentence'": <gensim.models.keyedvectors.Vocab object at 0x113a24a90>, "b'second'": <gensim.models.keyedvectors.Vocab object at 0x113a24c88>}


Say we want to further preprocess the words from the files — convert to unicode, lowercase, remove numbers, extract named entities… All of this can be done inside the `MySentences` iterator and `fasttext` doesn’t need to know. All that is required is that the input yields one sentence (list of utf8 words) after another.

**Note to advanced users:** calling `FastText(sentences, iter=1)` will run **two** passes over the sentences iterator. In general it runs `iter+1` passes. By the way, the default value is `iter=5` to comply with Facebooks's fasttext in c++ lang.

  1. The first pass collects words and their frequencies to build an internal dictionary tree structure. 
  2. The second pass trains the neural model.

These two passes can also be initiated manually, in case your input stream is non-repeatable (you can only afford one pass), and you’re able to initialize the vocabulary some other way:

In [34]:
# build the same model, making the 2 steps explicit
new_model = FastText(min_count=1)  # an empty model, no training
new_model.build_vocab(sentences)                 # can be a non-repeatable, 1-pass generator     
new_model.train(sentences, total_examples=new_model.corpus_count, epochs=new_model.iter)                       
# can be a non-repeatable, 1-pass generator

2018-06-16 17:41:19,108 : INFO : collecting all words and their counts
2018-06-16 17:41:19,110 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-16 17:41:19,111 : INFO : collected 3 word types from a corpus of 4 raw words and 4 sentences
2018-06-16 17:41:19,111 : INFO : Loading a fresh vocabulary
2018-06-16 17:41:19,112 : INFO : min_count=1 retains 3 unique words (100% of original 3, drops 0)
2018-06-16 17:41:19,113 : INFO : min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2018-06-16 17:41:19,114 : INFO : deleting the raw counts dictionary of 3 items
2018-06-16 17:41:19,115 : INFO : sample=0.001 downsamples 3 most-common words
2018-06-16 17:41:19,115 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2018-06-16 17:41:19,117 : INFO : estimated required memory for 3 words, 87 buckets and 100 dimensions: 39596 bytes
2018-06-16 17:41:19,118 : INFO : resetting layer weights
2018-06-16 17:41:19,569 : INFO : Total number of ng

In [35]:
print(new_model)
print(model.wv.vocab)

FastText(vocab=3, size=100, alpha=0.025)
{"b'first'": <gensim.models.keyedvectors.Vocab object at 0x113a24ac8>, "b'sentence'": <gensim.models.keyedvectors.Vocab object at 0x113a24a90>, "b'second'": <gensim.models.keyedvectors.Vocab object at 0x113a24c88>}


### More data would be nice
For the following examples, we'll use the [Lee Corpus](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/test/test_data/lee_background.cor) (which you already have if you've installed gensim):

In [36]:
# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [37]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

sentences = MyText()

print(sentences)

## Training
`FastText` accepts several parameters that affect both training speed and quality.

### min_count
`min_count` is for pruning the internal dictionary. Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them:

In [39]:
# default value of min_count=5
model = FastText(sentences, min_count=10)

2018-06-16 17:41:41,278 : INFO : collecting all words and their counts
2018-06-16 17:41:41,279 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-16 17:41:41,306 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-06-16 17:41:41,306 : INFO : Loading a fresh vocabulary
2018-06-16 17:41:41,316 : INFO : min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2018-06-16 17:41:41,318 : INFO : min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2018-06-16 17:41:41,324 : INFO : deleting the raw counts dictionary of 10186 items
2018-06-16 17:41:41,329 : INFO : sample=0.001 downsamples 54 most-common words
2018-06-16 17:41:41,330 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2018-06-16 17:41:41,457 : INFO : estimated required memory for 806 words, 8368 buckets and 100 dimensions: 4541208 bytes
2018-06-16 17:41:41,459 : INFO : resetting layer weights
2

### size
`size` is the number of dimensions (N) of the N-dimensional space that gensim FastTest maps the words onto.

Bigger size values require more training data, but can lead to better (more accurate) models. Reasonable values are in the tens to hundreds.

In [40]:
# default value of size=100
model = FastText(sentences, size=200)

2018-06-16 17:41:51,349 : INFO : collecting all words and their counts
2018-06-16 17:41:51,350 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-16 17:41:51,390 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-06-16 17:41:51,392 : INFO : Loading a fresh vocabulary
2018-06-16 17:41:51,416 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2018-06-16 17:41:51,418 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2018-06-16 17:41:51,427 : INFO : deleting the raw counts dictionary of 10186 items
2018-06-16 17:41:51,433 : INFO : sample=0.001 downsamples 49 most-common words
2018-06-16 17:41:51,437 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2018-06-16 17:41:51,628 : INFO : estimated required memory for 1723 words, 16397 buckets and 200 dimensions: 17078164 bytes
2018-06-16 17:41:51,630 : INFO : resetting layer weight

### workers
`workers`, the last of the major parameters (full list [here](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) is for training parallelization, to speed up training:

In [41]:
# default value of workers=3 (tutorial says 1...)
model = FastText(sentences, workers=4)

2018-06-16 17:41:59,702 : INFO : collecting all words and their counts
2018-06-16 17:41:59,704 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-16 17:41:59,730 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-06-16 17:41:59,731 : INFO : Loading a fresh vocabulary
2018-06-16 17:41:59,743 : INFO : min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2018-06-16 17:41:59,744 : INFO : min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2018-06-16 17:41:59,752 : INFO : deleting the raw counts dictionary of 10186 items
2018-06-16 17:41:59,754 : INFO : sample=0.001 downsamples 49 most-common words
2018-06-16 17:41:59,756 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2018-06-16 17:41:59,937 : INFO : estimated required memory for 1723 words, 16397 buckets and 100 dimensions: 9140964 bytes
2018-06-16 17:41:59,938 : INFO : resetting layer weights

The `workers` parameter only has an effect if you have [Cython](http://cython.org/) installed. Without Cython, you’ll only be able to use one core because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock) (and `fasttext` training will be [miserably slow](http://rare-technologies.com/word2vec-in-python-part-two-optimizing/)).

## Memory
At its core, `fasttext` model parameters are stored as matrices (NumPy arrays). Each array is **#vocabulary** (controlled by min_count parameter) times **#size** (size parameter) of floats (single precision aka 4 bytes).

Three such matrices are held in RAM (work is underway to reduce that number to two, or even one). So if your input contains 100,000 unique words, and you asked for layer `size=200`, the model will require approx. `100,000*200*4*3 bytes = ~229MB`.

There’s a little extra memory needed for storing the vocabulary tree (100,000 words would take a few megabytes), but unless your words are extremely loooong strings, memory footprint will be dominated by the three matrices above.

## Evaluating
`FastText` training is an unsupervised task, there’s no good way to objectively evaluate the result. Evaluation depends on your end application.

Google has released their testing set of about 20,000 syntactic and semantic test examples, following the “A is to B as C is to D” task. It is provided in the 'datasets' folder.

For example a syntactic analogy of comparative type is bad:worse;good:?. There are total of 9 types of syntactic comparisons in the dataset like plural nouns and nouns of opposite meaning.

The semantic questions contain five types of semantic analogies, such as capital cities (Paris:France;Tokyo:?) or family members (brother:sister;dad:?). 

Gensim supports the same evaluation set, in exactly the same format:

In [72]:
!wget -nc https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt

File 'questions-words.txt' already there; not retrieving.



In [42]:
model.accuracy('questions-words.txt')

/Users/joydeep/Documents/fastext_rnd/gensim_rnd/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `accuracy` (Method will be removed in 4.0.0, use self.wv.accuracy() instead).
  """Entry point for launching an IPython kernel.
2018-06-16 17:42:08,586 : INFO : precomputing L2-norms of word weight vectors
2018-06-16 17:42:08,588 : INFO : precomputing L2-norms of ngram weight vectors
2018-06-16 17:42:08,600 : INFO : family: 0.0% (0/2)
2018-06-16 17:42:08,624 : INFO : gram3-comparative: 16.7% (2/12)
2018-06-16 17:42:08,652 : INFO : gram4-superlative: 16.7% (2/12)
2018-06-16 17:42:08,683 : INFO : gram5-present-participle: 20.0% (4/20)
2018-06-16 17:42:08,722 : INFO : gram6-nationality-adjective: 10.0% (2/20)
2018-06-16 17:42:08,759 : INFO : gram7-past-tense: 0.0% (0/20)
2018-06-16 17:42:08,788 : INFO : gram8-plural: 16.7% (2/12)
2018-06-16 17:42:08,800 : INFO : total: 12.2% (12/98)


[{'section': 'capital-common-countries', 'correct': [], 'incorrect': []},
 {'section': 'capital-world', 'correct': [], 'incorrect': []},
 {'section': 'currency', 'correct': [], 'incorrect': []},
 {'section': 'city-in-state', 'correct': [], 'incorrect': []},
 {'section': 'family',
  'correct': [],
  'incorrect': [('HE', 'SHE', 'HIS', 'HER'), ('HIS', 'HER', 'HE', 'SHE')]},
 {'section': 'gram1-adjective-to-adverb', 'correct': [], 'incorrect': []},
 {'section': 'gram2-opposite', 'correct': [], 'incorrect': []},
 {'section': 'gram3-comparative',
  'correct': [('GOOD', 'BETTER', 'GREAT', 'GREATER'),
   ('LONG', 'LONGER', 'GREAT', 'GREATER')],
  'incorrect': [('GOOD', 'BETTER', 'LONG', 'LONGER'),
   ('GOOD', 'BETTER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'LONG', 'LONGER'),
   ('GREAT', 'GREATER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'LOW', 'LOWER'),
   ('LONG', 'LONGER', 'GOOD', 'BETTER'),
   ('LOW', 'LOWER', 'GOOD', 'BETTER'),
   ('LOW', 'LOWE

This `accuracy` takes an 
[optional parameter](http://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec.accuracy) `restrict_vocab` 
which limits which test examples are to be considered.



To ascertain semantic similarity one way is to use the academic dataset WS-353. This example is for illustration purposes and you should really be creating a similar dataset specific to your business. It contains word pairs together with human-assigned similarity judgments. It measures the relatedness or co-occurrence of two words. For example, 'coast' and 'shore' are very similar as they appear in the same context. At the same time 'clothes' and 'closet' are less similar because they are related but not interchangeable.

In [43]:
model.evaluate_word_pairs(test_data_dir + 'wordsim353.tsv')

/Users/joydeep/Documents/fastext_rnd/gensim_rnd/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `evaluate_word_pairs` (Method will be removed in 4.0.0, use self.wv.evaluate_word_pairs() instead).
  """Entry point for launching an IPython kernel.
2018-06-16 17:42:13,968 : INFO : Pearson correlation coefficient against /Users/joydeep/Documents/fastext_rnd/gensim_rnd/venv/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: -0.0730
2018-06-16 17:42:13,969 : INFO : Spearman rank-order correlation coefficient against /Users/joydeep/Documents/fastext_rnd/gensim_rnd/venv/lib/python3.6/site-packages/gensim/test/test_data/wordsim353.tsv: -0.0602
2018-06-16 17:42:13,970 : INFO : Pairs with unknown words ratio: 85.6%


((-0.0730136854705976, 0.6106289924879249),
 SpearmanrResult(correlation=-0.06018780418889944, pvalue=0.674812973833056),
 85.55240793201133)

Once again, **good performance on Google's or WS-353 test set doesn’t mean word2vec will work well in your application, or vice versa**. It’s always best to evaluate directly on your intended task. For an example of how to use word2vec in a classifier pipeline, see this [tutorial](https://github.com/RaRe-Technologies/movie-plots-by-genre).

## Storing and loading models
You can store/load models using the standard gensim methods:

In [44]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2018-06-16 17:42:17,821 : INFO : saving FastText object under /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp, separately None
2018-06-16 17:42:17,822 : INFO : not storing attribute vectors_norm
2018-06-16 17:42:17,823 : INFO : not storing attribute vectors_vocab_norm
2018-06-16 17:42:17,824 : INFO : not storing attribute vectors_ngrams_norm
2018-06-16 17:42:17,825 : INFO : not storing attribute buckets_word
2018-06-16 17:42:17,965 : INFO : saved /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp


In [45]:
new_model = gensim.models.FastText.load(temp_path)  # open the model

2018-06-16 17:42:20,145 : INFO : loading FastText object from /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp
2018-06-16 17:42:20,236 : INFO : loading wv recursively from /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp.wv.* with mmap=None
2018-06-16 17:42:20,237 : INFO : setting ignored attribute vectors_norm to None
2018-06-16 17:42:20,238 : INFO : setting ignored attribute vectors_vocab_norm to None
2018-06-16 17:42:20,239 : INFO : setting ignored attribute vectors_ngrams_norm to None
2018-06-16 17:42:20,240 : INFO : setting ignored attribute buckets_word to None
2018-06-16 17:42:20,240 : INFO : loading vocabulary recursively from /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp.vocabulary.* with mmap=None
2018-06-16 17:42:20,241 : INFO : loading trainables recursively from /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp.trainables.* with mmap=None
2018-06-16 17:42:20,242 : INFO : loaded /v

which uses pickle internally, optionally `mmap`‘ing the model’s internal large NumPy matrices into virtual memory directly from disk files, for inter-process memory sharing.

In addition, you can load models created by the original C tool, both using its text and binary formats:
```
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.txt', binary=False)
  # using gzipped/bz2 input works too, no need to unzip:
  model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/vectors.bin.gz', binary=True)
```

## Online training / Resuming training
Advanced users can load a model and continue training it with more sentences and [new vocabulary words](online_w2v_tutorial.ipynb):

In [46]:
model = gensim.models.FastText.load(temp_path)
more_sentences = [['Advanced', 'users', 'can', 'load', 'a', 'model', 'and', 'continue', 'training', 'it', 'with', 'more', 'sentences']]
model.build_vocab(more_sentences, update=True)
model.train(more_sentences, total_examples=model.corpus_count, epochs=model.iter)

# cleaning up temp
os.close(fs)
os.remove(temp_path)

2018-06-16 17:42:28,213 : INFO : loading FastText object from /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp
2018-06-16 17:42:28,305 : INFO : loading wv recursively from /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp.wv.* with mmap=None
2018-06-16 17:42:28,306 : INFO : setting ignored attribute vectors_norm to None
2018-06-16 17:42:28,307 : INFO : setting ignored attribute vectors_vocab_norm to None
2018-06-16 17:42:28,307 : INFO : setting ignored attribute vectors_ngrams_norm to None
2018-06-16 17:42:28,308 : INFO : setting ignored attribute buckets_word to None
2018-06-16 17:42:28,308 : INFO : loading vocabulary recursively from /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp.vocabulary.* with mmap=None
2018-06-16 17:42:28,309 : INFO : loading trainables recursively from /var/folders/qr/cd8zctlj28n379tty3w3mv4c0000gp/T/tmp7zxa8fgjgensim_temp.trainables.* with mmap=None
2018-06-16 17:42:28,310 : INFO : loaded /v

You may need to tweak the `total_words` parameter to `train()`, depending on what learning rate decay you want to simulate.

Note that it’s not possible to resume training with models generated by the C tool, `KeyedVectors.load_word2vec_format()`. You can still use them for querying/similarity, but information vital for training (the vocab tree) is missing there.

## Using the model
`FastText` supports several word similarity tasks out of the box:

In [47]:
model.most_similar(positive=['human', 'crime'], negative=['party'], topn=1)

/Users/joydeep/Documents/fastext_rnd/gensim_rnd/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2018-06-16 17:42:33,036 : INFO : precomputing L2-norms of word weight vectors
2018-06-16 17:42:33,038 : INFO : precomputing L2-norms of ngram weight vectors


[('prime', 0.9999166131019592)]

In [48]:
model.doesnt_match("input is lunch he sentence cat".split())

/Users/joydeep/Documents/fastext_rnd/gensim_rnd/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'lunch'

In [49]:
print(model.wv.similarity('human', 'party'))
print(model.wv.similarity('tree', 'murder'))

0.9999453137708809
0.9999373985788307


If you need the raw output vectors in your application, you can access these either on a word-by-word basis:

In [55]:
model.wv['tree']  # raw NumPy vector of a word

array([ 7.95634165e-02, -1.64566547e-01, -6.20548353e-02, -2.19143391e-01,
        5.39077930e-02,  3.49085927e-01, -8.31378400e-02,  1.04103945e-01,
       -2.29767680e-01,  8.64591356e-03, -8.27853158e-02,  1.60468429e-01,
       -1.68333769e-01, -1.63088441e-01,  7.42197931e-02, -1.09306023e-01,
       -1.68826491e-01, -2.24383563e-01, -3.52127990e-03, -2.15871528e-01,
        1.31302744e-01, -5.83257556e-01, -4.29152362e-02, -6.98756501e-02,
        1.39738768e-01,  6.58502728e-02, -2.78356761e-01, -3.34596634e-02,
        4.09523427e-01, -9.79609191e-02, -9.79696736e-02, -3.20060641e-01,
       -1.97024971e-01,  2.31504038e-01,  2.88332403e-01, -3.33068788e-01,
       -2.23926231e-01,  6.73641544e-03, -1.90017074e-01,  4.68985617e-01,
       -2.07753405e-02,  2.04322368e-01, -1.93995267e-01,  1.48907512e-01,
        3.58458832e-02, -1.38164014e-01,  3.63497555e-01, -1.99809343e-01,
       -3.48173887e-01,  3.09680939e-01, -9.27656144e-02, -7.81382918e-02,
       -2.34361082e-01, -

…or en-masse as a 2D NumPy matrix from `model.wv.syn0`.

## Training Loss Computation

The parameter `compute_loss` can be used to toggle computation of loss while training the Word2Vec model. The computed loss is stored in the model attribute `running_training_loss` and can be retrieved using the function `get_latest_training_loss` as follows : 

In [56]:
# instantiating and training the Word2Vec model
model_with_loss = FastText(sentences, min_count=1, hs=0, sg=1, seed=42)

model_with_loss.compute_loss = True

# getting the training loss value
training_loss = model_with_loss.running_training_loss
print(training_loss)

2018-06-16 17:47:51,339 : INFO : collecting all words and their counts
2018-06-16 17:47:51,341 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-06-16 17:47:51,366 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2018-06-16 17:47:51,367 : INFO : Loading a fresh vocabulary
2018-06-16 17:47:51,457 : INFO : min_count=1 retains 10186 unique words (100% of original 10186, drops 0)
2018-06-16 17:47:51,458 : INFO : min_count=1 leaves 59890 word corpus (100% of original 59890, drops 0)
2018-06-16 17:47:51,504 : INFO : deleting the raw counts dictionary of 10186 items
2018-06-16 17:47:51,509 : INFO : sample=0.001 downsamples 37 most-common words
2018-06-16 17:47:51,511 : INFO : downsampling leaves estimated 47231 word corpus (78.9% of prior 59890)
2018-06-16 17:47:52,439 : INFO : estimated required memory for 10186 words, 71608 buckets and 100 dimensions: 44273240 bytes
2018-06-16 17:47:52,443 : INFO : resetting layer weights
2

0.0


### Benchmarks to see effect of training loss compuation code on training time

We first download and setup the test data used for getting the benchmarks.

In [60]:
input_data_files = []

def setup_input_data():
    # check if test data already present
    if os.path.isfile('./text8') is False:

        # download and decompress 'text8' corpus
        import zipfile
#         ! wget 'http://mattmahoney.net/dc/text8.zip'
#         ! unzip 'text8.zip'
    
        # create 1 MB, 10 MB and 50 MB files
        ! head -c1000000 text8 > text8_1000000
        ! head -c10000000 text8 > text8_10000000
        ! head -c50000000 text8 > text8_50000000
                
    # add 25 KB test file
    input_data_files.append(os.path.join(os.getcwd(), 'gensim/gensim/test/test_data/lee_background.cor'))

    # add 1 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_1000000'))

    # add 10 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_10000000'))

    # add 50 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8_50000000'))

    # add 100 MB test file
    input_data_files.append(os.path.join(os.getcwd(), 'text8'))

setup_input_data()
print(input_data_files)

['/Users/joydeep/Documents/fastext_rnd/gensim_rnd/gensim/gensim/test/test_data/lee_background.cor', '/Users/joydeep/Documents/fastext_rnd/gensim_rnd/text8_1000000', '/Users/joydeep/Documents/fastext_rnd/gensim_rnd/text8_10000000', '/Users/joydeep/Documents/fastext_rnd/gensim_rnd/text8_50000000', '/Users/joydeep/Documents/fastext_rnd/gensim_rnd/text8']


We now compare the training time taken for different combinations of input data and model training parameters like `hs` and `sg`.

In [61]:
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# using 25 KB and 50 MB files only for generating o/p -> comment next line for using all 5 test files
input_data_files = [input_data_files[0], input_data_files[-2]]
print(input_data_files)

import time
import numpy as np
import pandas as pd

train_time_values = []
seed_val = 42
sg_values = [0, 1]
hs_values = [0, 1]

for data_file in input_data_files:
    data = LineSentence(data_file) 
    for sg_val in sg_values:
        for hs_val in hs_values:
            time_taken_list = []
            for i in range(3):
                start_time = time.time()
                w2v_model = FastText(data, sg=sg_val, hs=hs_val, seed=seed_val) 
                time_taken_list.append(time.time() - start_time)

            time_taken_list = np.array(time_taken_list)
            time_mean = np.mean(time_taken_list)
            time_std = np.std(time_taken_list)
            train_time_values.append({'train_data': data_file, 'sg': sg_val, 'hs': hs_val, 'mean': time_mean, 'std': time_std})

train_times_table = pd.DataFrame(train_time_values)
train_times_table = train_times_table.sort_values(by=['train_data', 'sg', 'hs'], ascending=[False, False, False])
print(train_times_table)

### Adding FastText "model to dict" method to production pipeline
Suppose, we still want more performance improvement in production. 
One good way is to cache all the similar words in a dictionary.
So that next time when we get the similar query word, we'll search it first in the dict.
And if it's a hit then we will show the result directly from the dictionary.
otherwise we will query the word and then cache it so that it doesn't miss next time.

In [63]:
logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)

In [64]:
most_similars_precalc = {word : model.wv.most_similar(word) for word in model.wv.index2word}
for i, (key, value) in enumerate(most_similars_precalc.items()):
    if i==3:
        break
    print(key, value)

the [('bonn', 0.9975321888923645), ('unit', 0.9975069761276245), ('dispute', 0.9974783062934875), ('50', 0.9974721074104309), ('israel,', 0.9974679946899414), ('envoy', 0.997464120388031), ('south-west', 0.9974561929702759), ('west', 0.9974554181098938), ('past', 0.9974445104598999), ('gaza', 0.9974406957626343)]
to [('to', 0.9999938607215881), ('resistance', 0.9999796748161316), ('several', 0.9999785423278809), ('told', 0.9999780058860779), ('seeking', 0.9999777674674988), ('regional', 0.9999773502349854), ('into', 0.9999769330024719), ('rest', 0.9999768733978271), ('tour', 0.9999761581420898), ('assisting', 0.9999760985374451)]
of [('of', 0.9999963045120239), ('offer', 0.9999821186065674), ('threatening', 0.9999819397926331), ('yesterday.', 0.9999812245368958), ('searching', 0.999981164932251), ('investigating', 0.999981164932251), ('injuring', 0.9999811053276062), ('operating', 0.9999808669090271), ('investigation', 0.9999807476997375), ('yesterday', 0.9999807476997375)]


### Comparison with and without caching

for time being lets take 4 words randomly

In [65]:
import time
words = ['voted','few','their','around']

Without caching

In [66]:
start = time.time()
for word in words:
    result = model.wv.most_similar(word)
    print(result)
end = time.time()
print(end-start)

[('voted', 0.9995853304862976), ('heavy', 0.9995341300964355), ('kandahar', 0.9995294809341431), ('crash', 0.9995188117027283), ('vote', 0.999514639377594), ('process', 0.9995137453079224), ('republic', 0.9995092749595642), ('peacekeepers', 0.999508261680603), ('be', 0.9995070099830627), ('verdict', 0.999505877494812)]
[('few', 0.9998751878738403), ('wake', 0.9996885061264038), ('although', 0.9996861815452576), ('anthony', 0.9996844530105591), ('reduce', 0.9996806383132935), ('though', 0.9996805787086487), ('andy', 0.9996801614761353), ('air', 0.9996786713600159), ('eve', 0.9996781945228577), ('thousands', 0.9996775984764099)]
[('their', 0.9999809861183167), ('flying', 0.9999514818191528), ('returning', 0.9999493956565857), ('training', 0.9999493360519409), ('making', 0.9999492168426514), ('responding', 0.9999487400054932), ('threatening', 0.9999486207962036), ('burning', 0.9999483823776245), ('going', 0.9999483227729797), ('alleged', 0.9999480843544006)]
[('around', 0.9999921917915344

Now with caching

In [67]:
start = time.time()
for word in words:
    if 'voted' in most_similars_precalc:
        result = most_similars_precalc[word]
        print(result)
    else:
        result = model.wv.most_similar(word)
        most_similars_precalc[word] = result
        print(result)
    
end = time.time()
print(end-start)

[('voted', 0.9995853304862976), ('heavy', 0.9995341300964355), ('kandahar', 0.9995294809341431), ('crash', 0.9995188117027283), ('vote', 0.999514639377594), ('process', 0.9995137453079224), ('republic', 0.9995092749595642), ('peacekeepers', 0.999508261680603), ('be', 0.9995070099830627), ('verdict', 0.999505877494812)]
[('few', 0.9998751878738403), ('wake', 0.9996885061264038), ('although', 0.9996861815452576), ('anthony', 0.9996844530105591), ('reduce', 0.9996806383132935), ('though', 0.9996805787086487), ('andy', 0.9996801614761353), ('air', 0.9996786713600159), ('eve', 0.9996781945228577), ('thousands', 0.9996775984764099)]
[('their', 0.9999809861183167), ('flying', 0.9999514818191528), ('returning', 0.9999493956565857), ('training', 0.9999493360519409), ('making', 0.9999492168426514), ('responding', 0.9999487400054932), ('threatening', 0.9999486207962036), ('burning', 0.9999483823776245), ('going', 0.9999483227729797), ('alleged', 0.9999480843544006)]
[('around', 0.9999921917915344

Clearly you can see the improvement but this difference will be even larger when we take more words in the consideration.

# Visualising the Word Embeddings

The word embeddings made by the model can be visualised by reducing dimensionality of the words to 2 dimensions using tSNE.

Visualisations can be used to notice semantic and syntactic trends in the data.

Example: Semantic- words like cat, dog, cow, etc. have a tendency to lie close by
         Syntactic- words like run, running or cut, cutting lie close together.
Vector relations like vKing - vMan = vQueen - vWoman can also be noticed.

Additional dependencies : 
- sklearn
- numpy
- plotly

The function below can be used to plot the embeddings in an ipython notebook.
It requires the model as the necessary parameter. If you don't have the model, you can load it by

`model = gensim.models.FastText.load('path/to/model')`

If you don't want to plot inside a notebook, set the `plot_in_notebook` parameter to `False`.

Note: the model used for the visualisation is trained on a small corpus. Thus some of the relations might not be so clear

Beware : This sort dimension reduction comes at the cost of loss of information.

In [70]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

def reduce_dimensions(model, plot_in_notebook = True):

    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = []        # positions in vector space
    labels = []         # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)


    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    # reduce using t-SNE
    vectors = np.asarray(vectors)
    logging.info('starting tSNE dimensionality reduction. This may take some time.')
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
        
    # Create a trace
    trace = go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='text',
        text=labels
        )
    
    data = [trace]
    
    logging.info('All done. Plotting.')
    
    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

In [71]:
reduce_dimensions(model)

## Conclusion

In this tutorial we learned how to train fasttext models on your custom data and also how to evaluate it. Hope that you too will find this popular tool useful in your Machine Learning tasks!



# Links

* P. Bojanowski*, E. Grave*, A. Joulin, T. Mikolov, [Enriching Word Vectors with Subword Information](https://arxiv.org/abs/1607.04606).
* A. Joulin, E. Grave, P. Bojanowski, T. Mikolov, [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759).
* A. Joulin, E. Grave, P. Bojanowski, M. Douze, H. Jégou, T. Mikolov, [FastText.zip: Compressing text classification models](https://arxiv.org/abs/1612.03651)
* [Gensim fasttext tutorial](https://radimrehurek.com/gensim/models/fasttext.html#module-gensim.models.fasttext). 